In [20]:
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, Union
from stable_baselines3.common.type_aliases import GymStepReturn

import gymnasium as gym
import numpy as np
import pytest
from gymnasium import spaces
from gymnasium.spaces.space import Space


from stable_baselines3 import SAC, A2C, PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.sac.custom_policies import CustomActor, CustomContinousCritic, CustomSACPolicy
from stable_baselines3.ppo.custom_policies import CustomActorCriticPolicy, CustomNetwork, CustomFeaturesExtractor
import torch

ImportError: cannot import name 'CustomFeaturesExtractor' from 'stable_baselines3.ppo.custom_policies' (/home/irobotics/stable-baselines3/stable_baselines3/ppo/custom_policies.py)

In [11]:
class FakeImageEnv(gym.Env):
    """
    Fake image environment for testing purposes, it mimics Atari games.

    :param action_dim: Number of discrete actions
    :param screen_height: Height of the image
    :param screen_width: Width of the image
    :param n_channels: Number of color channels
    :param discrete: Create discrete action space instead of continuous
    :param channel_first: Put channels on first axis instead of last
    """

    def __init__(
        self,
        action_dim: int = 6,
        screen_height: int = 64,
        screen_width: int = 64,
        n_channels: int = 3,
        discrete: bool = False,
        channel_first: bool = False,
    ) -> None:
        self.observation_shape = (screen_height, screen_width, n_channels)
        if channel_first:
            self.observation_shape = (n_channels, screen_height, screen_width)
        self.observation_space = spaces.Box(low=0, high=255, shape=self.observation_shape, dtype=np.uint8)
        if discrete:
            self.action_space = spaces.Discrete(action_dim)
        else:
            self.action_space = spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        self.ep_length = 10
        self.current_step = 0

    def reset(self, *, seed: Optional[int] = None, options: Optional[Dict] = None) -> Tuple[np.ndarray, Dict]:
        if seed is not None:
            super().reset(seed=seed)
        self.current_step = 0
        return self.observation_space.sample(), {}

    def step(self, action: Union[np.ndarray, int]) -> GymStepReturn:
        reward = 0.0
        self.current_step += 1
        terminated = False
        truncated = self.current_step >= self.ep_length
        return self.observation_space.sample(), reward, terminated, truncated, {}

    def render(self, mode: str = "human") -> None:
        pass

In [12]:
inp = torch.randn(1,3,64,64).to("cuda")


In [4]:
model = PPO(CustomActorCriticPolicy, env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/irobotics/stable-baselines3/sb3-dev/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/irobotics/stable-baselines3/sb3-dev/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
env = FakeImageEnv(discrete=True)
policy_kwargs = dict(
    features_extractor_class=CustomFeaturesExtractor,
    # features_extractor_kwargs=dict(features_dim=128),
)
model = PPO(CustomActorCriticPolicy, env, verbose=1, policy_kwargs=policy_kwargs)

NameError: name 'CustomFeaturesExtractor' is not defined

In [15]:
model.policy.features_extractor_class = CustomFeaturesExtractor
model.__init__()

NameError: name 'CustomFeaturesExtractor' is not defined

In [9]:
model.policy(inp)

(tensor([5], device='cuda:0'),
 tensor([[-0.0103]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([-1.7910], device='cuda:0', grad_fn=<SqueezeBackward1>))

In [6]:
model.learn(total_timesteps=100)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 1114     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [19]:
a, b = model.policy.mlp_extractor(torch.randn(1,3,64,64).to("cuda"))

In [28]:
model.policy.features_extractor(inp)

tensor([[3.6942, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3737,
         0.4267, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6754,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.8732, 0.0000, 0.0000, 1.6379,
         0.0000, 0.0000, 0.0000, 1.5004, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.5894, 1.4374, 0.7901, 0.0000, 0.0000, 0.0000, 0.8521, 0.1529, 0.0000,
         0.3006, 0.0000, 0.0000, 0.1547, 0.0000, 0.0000, 0.0000, 0.0000, 1.8882,
         0.0000, 0.8816, 0.7326, 0.6156, 2.6843, 0.1397, 0.0000, 0.0000, 1.0876,
         0.0000, 0.0000, 0.0000, 0.0000, 1.9841, 0.0000, 1.4970, 0.8682, 1.1011,
         0.0000, 0.6987, 1.1340, 0.0000, 3.7269, 2.3969, 0.0000, 0.0000, 0.4092,
         1.3309, 0.0000, 1.9305, 0.0000, 0.0000, 1.8371, 0.0000, 0.4794, 1.7046,
         0.0572, 0.1056, 0.9772, 1.6199, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.7156, 0.0000, 0.0000, 0.0000, 0.4432, 0.0000,
         0.0000, 1.0726, 1.7

In [26]:
model.policy(inp)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (192x64 and 512x1)

In [27]:
model#.policy.mlp_extractor.policy_net.forward(torch.randn(1,3,64,64).to("cuda"))[0].shape

In [21]:
model = SAC("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=1000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 1209     |
|    time_elapsed    | 0        |
|    total_timesteps | 40       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 1213     |
|    time_elapsed    | 0        |
|    total_timesteps | 80       |
---------------------------------


/home/irobotics/stable-baselines3/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 24.61GB > 19.96GB
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 12       |
|    fps             | 135      |
|    time_elapsed    | 0        |
|    total_timesteps | 120      |
| train/             |          |
|    actor_loss      | -7.35    |
|    critic_loss     | 0.137    |
|    ent_coef        | 0.995    |
|    ent_coef_loss   | -0.0546  |
|    learning_rate   | 0.0003   |
|    n_updates       | 19       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    episodes        | 16       |
|    fps             | 81       |
|    time_elapsed    | 1        |
|    total_timesteps | 160      |
| train/             |          |
|    actor_loss      | -8.22    |
|    critic_loss     | 0.118    |
|    ent_coef 

<function stable_baselines3.common.utils.constant_fn.<locals>.func(_)>